In [1]:
import os
from backoff import on_exception, expo
import json
import requests
from collections import Counter
from datetime import datetime
from ratelimit import limits, sleep_and_retry, RateLimitException
import time
import urllib.request
import asyncio
import aiohttp
import nest_asyncio
from async_timeout import timeout
from requests.exceptions import HTTPError

In [2]:
data_path = "D:\\METIS\\data\\"
image_path = "D:\\METIS\\images\\"

images_saved = []
met_objects = []

In [3]:
url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects'
response = requests.get(url, allow_redirects=True)
all_objects = json.loads(response.text)
objectIDs = all_objects['objectIDs']

In [4]:
@sleep_and_retry
@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=300, period=300)
def get_object_details(object_id, img_url=True):
    url ='https://collectionapi.metmuseum.org/public/collection/v1/objects/{}'.format(object_id)
    response = requests.get(url, timeout=10)
    response_json = json.loads(response.text)
    met_objects.append(response_json)
    
    if img_url:
        return response_json['primaryImage']

@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=300, period=300)
def get_image(object_id, url):
    filename = image_path + "{}.jpg".format(object_id)
    im = urllib.request.urlretrieve(url, filename)
    images_saved.append({
                        'objectID': object_id, 
                        'url': url,
                        'filename': filename
                        })

In [5]:
img_url = get_object_details(1567)
get_image(1567, img_url)

In [6]:
images_saved

[{'objectID': 1567,
  'url': 'https://images.metmuseum.org/CRDImages/ad/original/79684.jpg',
  'filename': 'D:\\METIS\\images\\1567.jpg'}]

In [7]:
images_saved = []
met_objects = []

start = datetime.now()

for i, object_id in enumerate(objectIDs):
    if i % 1000 == 0:
        with open(data_path + "images_saved.json", "w") as write_file:
            json.dump(images_saved, write_file)   
            
        with open(data_path + "met_objects.json", "w") as write_file:
            json.dump(met_objects, write_file)  
            
        print("{}: {} elapsed".format(i, datetime.now() - start))
            
    try:
        img_url = get_object_details(object_id)
        get_image(object_id, img_url)
        
    except:
        pass
   # print(i, object_id)

In [8]:
start = datetime.now()

for i, object_id in enumerate(objectIDs[420000:440000]):
    if i % 1000 == 0:
        with open(data_path + "images_saved_420.json", "w") as write_file:
            json.dump(images_saved, write_file)   
            
        with open(data_path + "met_objects_420.json", "w") as write_file:
            json.dump(met_objects, write_file)  
            
        print("{}: {} elapsed".format(i + 420000, datetime.now() - start))
            
    try:
        img_url = get_object_details(object_id)
        get_image(object_id, img_url)
        
    except:
        pass

420000: 0:00:00.024997 elapsed
421000: 0:15:31.639912 elapsed
422000: 0:31:13.812632 elapsed
423000: 0:50:37.093434 elapsed
424000: 1:06:11.984972 elapsed
425000: 1:23:04.420415 elapsed
426000: 1:41:22.551656 elapsed
427000: 1:57:13.916746 elapsed
428000: 2:13:22.784426 elapsed
429000: 2:31:43.321548 elapsed
430000: 2:50:58.165857 elapsed
431000: 3:10:14.410173 elapsed
432000: 3:32:36.597832 elapsed
433000: 3:54:51.002076 elapsed
434000: 4:13:19.615766 elapsed
435000: 4:30:30.006348 elapsed
436000: 4:45:53.071558 elapsed
437000: 5:02:59.120399 elapsed
438000: 5:22:21.219627 elapsed
439000: 5:38:37.422707 elapsed


In [9]:
with open(data_path + "images_saved_420.json", "w") as write_file:
    json.dump(images_saved, write_file)   
            
with open(data_path + "met_objects_420.json", "w") as write_file:
    json.dump(met_objects, write_file)  

In [10]:
len(images_saved)

9562